In [1]:
cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [21]:
pwd

'C:\\Users\\OLOYEDE_O\\PycharmProjects\\scraper'

In [22]:
import getpass
import datetime
import time
import json
import sys
import requests
import pandas as pd
from iea_scraper.settings import PROXY_DICT, SSL_CERTIFICATE_PATH
import logging

In [23]:
#set the logging parameters
logging.basicConfig(level=logging.DEBUG)

In [24]:
#{'Measurecodes':['L'],'ProductTypeCode':'WMC','VariableCOde':'GDPPPP'},{'Measurecodes':['L'],'ProductTypeCode':'WMC','VariableCOde':'RXD'},{'Measurecodes':['L'],'ProductTypeCode':'WMC','VariableCOde':'PGDP'}]

In [70]:
sample_selection_one = {
    'DatabankCode': 'WDMacro',
    'Frequency': 'Both',
    'GroupingMode': 'false',
    'IndicatorSortOrder': 'AlphabeticalOrder',
    'IsTemporarySelection': 'true',
    'ListingType': 'Private',
    'LocationSortOrder': 'AlphabeticalOrder',
    'Order': 'IndicatorLocation',
    'Sequence': 'EarliestToLatest',
    'StackedQuarters': 'true',
    'StartYear': 2010,
    'EndYear': '',

    # note: the fields below have been assigned empty lists
    'Regions': [
    ],
    'Variables': [{'Measurecodes':['L'],'ProductTypeCode':'WMC','VariableCOde':'GVAIND'}]}

In [71]:
sample_selection_one

{'DatabankCode': 'WDMacro',
 'Frequency': 'Both',
 'GroupingMode': 'false',
 'IndicatorSortOrder': 'AlphabeticalOrder',
 'IsTemporarySelection': 'true',
 'ListingType': 'Private',
 'LocationSortOrder': 'AlphabeticalOrder',
 'Order': 'IndicatorLocation',
 'Sequence': 'EarliestToLatest',
 'StackedQuarters': 'true',
 'StartYear': 2010,
 'EndYear': '',
 'Regions': [],
 'Variables': [{'Measurecodes': ['L'],
   'ProductTypeCode': 'WMC',
   'VariableCOde': 'GVAIND'}]}

In [72]:
API_KEY = 'cb-f337b53c-2279-47c5-928a-5a9b6c11a15d' # insert api key
BASE_URL = 'https://services.oxfordeconomics.com'


def databank_download(selection_dictionary):
    headers = {'Accept': 'application/json',
        'Api-Key': API_KEY,
        'Content-Type': 'application/json; charset=utf8' }

    def _download_url(base_url, page, page_size):
        url = base_url + '/api/download?includemetadata=true'
        return url + '&page={1}+&pagesize={2}'.format(base_url, page, page_size)

    page = 0
    page_size = 5000
    data = None

    while True:
        # note: _download_url returns a link of the form
        # ../api/download?includemetadata=true&page={page}+&page_size={page_size}
        response = requests.post(_download_url(BASE_URL, page, page_size),
            headers=headers,
            data=json.dumps(selection_dictionary),verify = SSL_CERTIFICATE_PATH)
        
        new_data = response.json()
        page += 1

        if data is None:
            data = new_data
        else:
            data.extend(new_data)

        if len(new_data) < page_size:
            break
            
    return data

In [73]:
data = databank_download(sample_selection_one)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): services.oxfordeconomics.com:443
DEBUG:urllib3.connectionpool:https://services.oxfordeconomics.com:443 "POST /api/download?includemetadata=true&page=0+&pagesize=5000 HTTP/1.1" 200 282338


In [74]:
len(data)

177

In [75]:
df = pd.json_normalize(data)

In [76]:
to_delete = ['Metadata.DatabankName','Metadata.AuthorTelephone','Metadata.IncomeBandName','Metadata.MarketSectorName','Metadata.CommodityName','Metadata.ScenarioName','Metadata.PartnerName','Metadata.AnnualTypeCode','Metadata.SourceDetails','Quarter','Metadata.Author','Metadata.ImposedEndYear','Metadata.ImposedEndQuarter']

In [77]:
df.drop(labels=to_delete,axis=1,inplace=True)

In [78]:
id_ = ['DatabankCode','ProductTypeCode', 'LocationCode', 'VariableCode','MeasureCode','Metadata.Description', 'Metadata.Location', 'Metadata.ScaleFactor','Metadata.AuthorEmail','Metadata.HistoricalEndYear','Metadata.HistoricalEndQuarter','Metadata.BaseYearPrice', 'Metadata.LastUpdate', 'Metadata.SeasonallyAdjusted','Metadata.SectorCoverage', 'Metadata.BaseYearIndex', 'Metadata.Units','Metadata.Source', 'Metadata.AdditionalSourceDetails','Metadata.MeasureName', 'Metadata.IndicatorName','Metadata.HasQuarterly', 'Metadata.CategoryDescription']

In [79]:
# melt the dataframe into the preferred shape
df = df.melt(id_vars=id_,value_name='value')

In [80]:
# rename the columns
df.rename(columns=lambda x:x.replace('Metadata.',''),inplace=True)

In [81]:
# split the column
df[['Quarterly/Annual','period']] = df['variable'].str.split('.',expand = True)
df.drop('variable',axis=1,inplace = True)

In [82]:
df = df[df['value'].notna()]

In [83]:
df.columns = df.columns.str.lower()

In [84]:
df.head()

,databankcode,producttypecode,locationcode,variablecode,measurecode,description,location,scalefactor,authoremail,historicalendyear,...,units,source,additionalsourcedetails,measurename,indicatorname,hasquarterly,categorydescription,value,quarterly/annual,period
0,WDMacro,WMC,AFG,GVAIND,L,"GVA in industry, real, LCU",Afghanistan,Millions: 2015 prices,thunter@oxfordeconomics.com,2020.0,...,Afghani,World Bank / Haver Analytics,Afghanistan: Industry including Construction: ...,Level,"GVA in industry, real, LCU",False,GDP and domestic demand,205881.0,AnnualData,2010
1,WDMacro,WMC,ALB,GVAIND,L,"Gross value added in industry, real, LCU",Albania,Millions: chained 2010 prices,rfennessy@oxfordeconomics.com,2020.0,...,Lek,World Bank/Haver Analytics,"Albania: Industry, Value Added (Mil.ALL) in Na...",Level,"Gross value added in industry, real, LCU",False,GDP and domestic demand,309136.0,AnnualData,2010
2,WDMacro,WMC,DZA,GVAIND,L,"Gross value added in industry, real, LCU",Algeria,Millions: 2005 prices,pvaneck@oxfordeconomics.com,2017.0,...,Dinar,World Bank\NSO Algeria,,Level,"Gross value added in industry, real, LCU",False,GDP and domestic demand,4230500.0,AnnualData,2010
3,WDMacro,WMC,AGO,GVAIND,L,"Gross value added in industry, real, LCU",Angola,Millions: 2002 prices,gvanrooyen@oxfordeconomics.com,2020.0,...,Kwanza,World Bank\Haver Analytics,Angola: Industry including Construction: Value...,Level,"Gross value added in industry, real, LCU",False,GDP and domestic demand,754968.0,AnnualData,2010
4,WDMacro,WMC,ATG,GVAIND,L,"Gross value added in industry, real, LCU",Antigua and Barbuda,,yadwani@oxfordeconomics.com,2020.0,...,EC$,Oxford Economics,,Level,"Gross value added in industry, real, LCU",False,GDP and domestic demand,0.0,AnnualData,2010


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4741 entries, 0 to 11163
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   databankcode             4741 non-null   object 
 1   producttypecode          4741 non-null   object 
 2   locationcode             4741 non-null   object 
 3   variablecode             4741 non-null   object 
 4   measurecode              4741 non-null   object 
 5   description              4741 non-null   object 
 6   location                 4741 non-null   object 
 7   scalefactor              4741 non-null   object 
 8   authoremail              4741 non-null   object 
 9   historicalendyear        4741 non-null   int64  
 10  historicalendquarter     4741 non-null   int64  
 11  baseyearprice            4741 non-null   object 
 12  lastupdate               4741 non-null   object 
 13  seasonallyadjusted       4741 non-null   bool   
 14  sectorcoverage         

In [ ]:
df.period.unique()

In [ ]:
df.head()

In [ ]:
df[df["VariableCode"]== 'GDPPPP']

In [ ]:
df.info()

In [ ]:
code_names = {'GDPPP': 'GDP,PPP exchange rate', 'RXD': 'Exchange rate,average period', 'PGDP': 'GDP deflator'}

In [ ]:
code_names

In [ ]:
for i in code_names.items():
    print(i[])

In [ ]:
sample_selection_one

In [ ]:
code_names = {'GDPPP': {'Measurecodes': ['L'], 'ProductTypeCode': 'WMC', 'VariableCOde': 'GDPPPP'},
                  'RXD': {'Measurecodes': ['L'], 'ProductTypeCode': 'WMC', 'VariableCOde': 'RXD'},
                  'PGDP': {'Measurecodes': ['L'], 'ProductTypeCode': 'WMC', 'VariableCOde': 'PGDP'}}

In [ ]:
for i in code_names.keys():
    sample_selection_one['Olaoye'] = code_names[i]
    print(sample_selection_one)
    print()

In [1]:
cd ..

C:\Users\OLOYEDE_O\PycharmProjects\scraper


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
import sys

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)

In [4]:
# from scraper.jobs.ng_gov_dpr.nigerian_oil_supply import NigerianOilSupplyJob

from iea_scraper.core import factory

# job = NigerianOilSupplyJob()
# job = factory.get_scraper_job('ng_gov_dpr', 'nigerian_oil_supply')

# job = NigerianOilSupplyJob(full_load=True)
job = factory.get_scraper_job('com_oxfordeconomics', 'oxford_economics_api',full_load = False)

2022-01-21 19:24:52,137 - iea_scraper.core.factory - DEBUG - Loading module iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api
2022-01-21 19:24:52,141 - iea_scraper.core.factory - DEBUG - Getting class OxfordEconomicsApiJob
2022-01-21 19:24:52,142 - iea_scraper.core.job - INFO - Temporary table name: #oxford_economics_api_temp, final table name: oxford_economics_api_data


In [5]:
job.run()

2022-01-21 19:24:56,143 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding sources to the basesource object
2022-01-21 19:24:56,143 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - 8 sources was successfully added to the list of sources
2022-01-21 19:24:56,144 - iea_scraper.core.job - DEBUG - download: True, parallel download: True
2022-01-21 19:24:56,144 - iea_scraper.core.utils - INFO - Executing function download_source() over 8 items with a maximum of 15 parallel workers.
2022-01-21 19:24:56,146 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:24:56,149 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:24:56,156 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:24:56,159 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:24:56,162 

In [5]:
job.get_sources()

2022-01-21 19:08:48,811 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding sources to the basesource object
2022-01-21 19:08:48,812 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - 8 sources was successfully added to the list of sources


In [ ]:
job.get_sources()

In [6]:
[vars(s) for s in job.sources]

[{'code': 'com_oxfordeconomics_GDPPPP',
  'url': 'https://services.oxfordeconomics.com',
  'path': 'com_oxfordeconomics_GDPPPP.json',
  'long_name': 'Oxford_Economics_Api_Data_for_GDPPPP',
  'meta_data': {'input_data': {'DatabankCode': 'WDMacro',
    'Frequency': 'Both',
    'GroupingMode': 'false',
    'IndicatorSortOrder': 'AlphabeticalOrder',
    'IsTemporarySelection': 'true',
    'ListingType': 'Private',
    'LocationSortOrder': 'AlphabeticalOrder',
    'Order': 'IndicatorLocation',
    'Sequence': 'EarliestToLatest',
    'StackedQuarters': 'true',
    'StartYear': 2000,
    'EndYear': 2030,
    'Regions': [],
    'Variables': [{'Measurecodes': ['L'],
      'ProductTypeCode': 'WMC',
      'VariableCode': 'GDPPPP'}]}}},
 {'code': 'com_oxfordeconomics_RXD',
  'url': 'https://services.oxfordeconomics.com',
  'path': 'com_oxfordeconomics_RXD.json',
  'long_name': 'Oxford_Economics_Api_Data_for_RXD',
  'meta_data': {'input_data': {'DatabankCode': 'WDMacro',
    'Frequency': 'Both',
  

In [7]:
job.download_and_get_checksum()

2022-01-21 19:09:53,784 - iea_scraper.core.job - DEBUG - download: True, parallel download: True
2022-01-21 19:09:53,785 - iea_scraper.core.utils - INFO - Executing function download_source() over 8 items with a maximum of 15 parallel workers.
2022-01-21 19:09:53,786 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,789 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,793 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,797 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,800 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,805 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:09:53,808 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api 

In [8]:
job.transform()

2022-01-21 19:10:11,924 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding data to the dataframe from com_oxfordeconomics_GDPPPP.json
2022-01-21 19:10:11,925 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - processing file with name com_oxfordeconomics_GDPPPP.json
2022-01-21 19:10:12,175 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding data to the dataframe from com_oxfordeconomics_RXD.json
2022-01-21 19:10:12,176 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - processing file with name com_oxfordeconomics_RXD.json
2022-01-21 19:10:12,366 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding data to the dataframe from com_oxfordeconomics_PGDP.json
2022-01-21 19:10:12,366 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - processing file with name com_oxfordeconomics_PGDP.json
2022-01-21 19:10:12,596 - iea_scraper.jobs.com_oxfordeconomics.oxford_econ

In [9]:
job.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121693 entries, 0 to 29135
Data columns (total 26 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   databankcode             121693 non-null  object 
 1   producttypecode          121693 non-null  object 
 2   locationcode             121693 non-null  object 
 3   variablecode             121693 non-null  object 
 4   measurecode              121693 non-null  object 
 5   description              121693 non-null  object 
 6   location                 121693 non-null  object 
 7   scalefactor              121693 non-null  object 
 8   authoremail              121693 non-null  object 
 9   historicalendyear        121451 non-null  float64
 10  historicalendquarter     121451 non-null  float64
 11  baseyearprice            121693 non-null  object 
 12  lastupdate               121693 non-null  object 
 13  seasonallyadjusted       121693 non-null  bool   
 14  secto

In [ ]:
job.data.reset_index(drop=True,inplace=True)

In [13]:
job.run()

2022-01-21 19:12:58,695 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - adding sources to the basesource object
2022-01-21 19:12:58,697 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - 8 sources was successfully added to the list of sources
2022-01-21 19:12:58,697 - iea_scraper.core.job - DEBUG - download: True, parallel download: True
2022-01-21 19:12:58,698 - iea_scraper.core.utils - INFO - Executing function download_source() over 8 items with a maximum of 15 parallel workers.
2022-01-21 19:12:58,699 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:12:58,701 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-21 19:12:58,704 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): services.oxfordeconomics.com:443
2022-01-21 19:12:58,704 - iea_scraper.jobs.com_oxfordeconomics.oxford_economics_api - DEBUG - reading page 0...
2022-01-2

In [12]:
job.data['variablecode'].unique()

array(['GDPPPP', 'RXD', 'PGDP', 'RXDPPP', 'CSA', 'GVAIND', 'IP', 'GVAAGR'],
      dtype=object)

In [ ]:
job.data.index()

In [ ]:
job.data.set_index('VariableCode')